In [16]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO
from tensorflow.keras.models import load_model
classifier = load_model('model/tl_classifier.h5')
from keras.preprocessing import image

In [17]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [18]:
def predict_image(img):
    test_image=cv2.resize(img, (64, 64),
                       interpolation=cv2.INTER_CUBIC)
    test_image = np.array(test_image, dtype='float32')
    test_image=np.expand_dims(test_image,axis=0)
    result=classifier.predict(test_image)
    return result

In [19]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [20]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
    
        if cl == 9:
            crop_area = image[left:bottom,top:right]
            result=predict_image(crop_area)
            if result[0][0]==1:
                cv2.rectangle(image, (top, left), (right, bottom), (0, 0, 255), 2)
                cv2.putText(image, 'Red - Stop',(top, left - 6),cv2.FONT_HERSHEY_SIMPLEX,0.8, (0, 0, 255), 2,cv2.LINE_AA)
            elif result[0][0]==0:
                cv2.rectangle(image, (top, left), (right, bottom), (0, 255, 0), 2)
                cv2.putText(image, 'Green - Proceed',
                        (top, left - 6),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0, 255, 0), 2,
                        cv2.LINE_AA)
        else:
            continue

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [21]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [22]:

def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while camera.isOpened():
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(30) & 0xff == ord('q'):
                break

    vout.release()
    camera.release()
    cv2.destroyAllWindows()

In [23]:

# if __name__ == '__main__':
#     yolo = YOLO(0.6, 0.5)
#     file = 'data/coco_classes.txt'
#     all_classes = get_classes(file)


yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

In [27]:
# detect images in test floder.
for (root, dirs, files) in os.walk('images/test'):
    if files:
        for f in files:
            print(f)
            path = os.path.join(root, f)
            image = cv2.imread(path)
            image = detect_image(image, yolo, all_classes)
            cv2.imwrite('images/res/' + f, image)

t1.jpg
time: 0.54s
class: traffic light, score: 1.00
box coordinate x,y,w,h: [118.64209771  23.78627068 185.91453454 374.70433293]
class: traffic light, score: 0.76
box coordinate x,y,w,h: [420.76812959 142.55411124  73.45008384 113.89437232]



In [29]:
video = 'T2.mp4'
detect_video(video, yolo, all_classes)

time: 0.60s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 557.66942739  120.02800941  295.67233711 1378.91917155]

time: 2.39s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 558.75108719  123.82089615  292.99563468 1368.91639709]

time: 1.37s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 558.90764236  133.53429794  291.53478563 1346.18704759]

time: 0.93s
class: traffic light, score: 0.92
box coordinate x,y,w,h: [ 558.20134163  125.77208519  293.70310217 1358.52590891]

time: 0.87s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 558.47286701  116.54992819  291.95679903 1383.46737275]

time: 0.88s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 558.57985497  112.68482208  294.27777618 1385.98342675]

time: 0.81s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 556.34533882  120.73984623  297.45890558 1372.52314788]

time: 0.86s
class: traffic light, score: 0.94
box coordinate x,y,w,h: [ 559.06361818  118.5642886

time: 0.60s
class: traffic light, score: 0.67
box coordinate x,y,w,h: [ 559.00330067   78.35810423  285.57667673 1410.00718887]

time: 0.60s
class: traffic light, score: 0.61
box coordinate x,y,w,h: [ 558.61178398   77.7513957   286.63144469 1409.8733029 ]

time: 0.60s
time: 0.66s
time: 0.63s
time: 0.61s
time: 0.59s
time: 0.61s
time: 0.60s
class: traffic light, score: 0.68
box coordinate x,y,w,h: [ 556.10966921   68.51084232  286.50671393 1418.6531287 ]

time: 0.58s
class: traffic light, score: 0.63
box coordinate x,y,w,h: [ 554.32402611   64.14175987  289.10327196 1430.18834738]

time: 0.63s
class: traffic light, score: 0.71
box coordinate x,y,w,h: [ 554.87235546   64.09054756  287.88148016 1427.58470645]

time: 0.60s
class: traffic light, score: 0.66
box coordinate x,y,w,h: [ 555.54878712   67.50149488  287.44800031 1427.03107871]

time: 0.61s
time: 0.60s
time: 0.61s
time: 0.62s
time: 0.59s
time: 0.64s
time: 0.61s
time: 0.59s
time: 0.62s
time: 0.58s
time: 0.62s
time: 0.61s
time: 0.60

time: 0.59s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 549.00507689  127.72406101  310.04092544 1284.58043685]

time: 0.61s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 547.19632387  128.71307373  315.56859612 1276.92867389]

time: 0.67s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.37136459  136.77450657  309.36795384 1264.86673539]

time: 0.66s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.28407478  131.51136875  306.39824152 1273.85790275]

time: 0.66s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.43621778  132.54002094  305.46267629 1270.71261736]

time: 0.65s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.35137415  137.09571362  304.71899897 1266.06179567]

time: 0.63s
class: traffic light, score: 0.96
box coordinate x,y,w,h: [ 551.52714729  126.262393    300.43465286 1284.04674493]

time: 0.63s
class: traffic light, score: 0.95
box coordinate x,y,w,h: [ 552.65824556  125.3589105

time: 0.62s
class: traffic light, score: 0.98
box coordinate x,y,w,h: [ 551.50191307  109.24540758  289.611682   1309.63247373]

time: 0.61s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 551.3582325   111.45441055  289.05018836 1306.32693914]

time: 0.58s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 551.76056385  109.63342667  288.82921189 1310.87219899]

time: 0.61s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 551.8351078   116.49812937  287.94788897 1301.07063477]

time: 0.59s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.87987661  117.29108334  288.22892547 1304.78665132]

time: 0.61s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 551.05220318  123.23747635  285.86454481 1292.54801383]

time: 0.59s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 551.00199223  125.41477203  286.12332433 1284.26331557]

time: 0.62s
class: traffic light, score: 0.97
box coordinate x,y,w,h: [ 550.73729038  127.3340892